<a href="https://colab.research.google.com/github/FaizKhalifah/UNB-CIC-IOT-2023-Dataset-classification/blob/main/Naive_Bayes_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import dataset

In [ ]:
from google.colab import files
upload = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
! kaggle datasets download -d madhavmalhotra/unb-cic-iot-dataset

Dataset URL: https://www.kaggle.com/datasets/madhavmalhotra/unb-cic-iot-dataset
License(s): other
100% 2.77G/2.77G [00:39<00:00, 83.7MB/s]
100% 2.77G/2.77G [00:39<00:00, 74.7MB/s]


In [ ]:
! chmod 600 /content/kaggle.json

In [ ]:
! unzip unb-cic-iot-dataset -d data_siap_pakai

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
rootFolder = 'data_siap_pakai/'
dataFolder = rootFolder+"wataiData/"
csvFolder = dataFolder+"csv/"
datasetFolder = csvFolder+"datasets/"

## Membaca Dataset

In [ ]:
df_sets = [k for k in os.listdir(datasetFolder) if k.endswith('.csv')]
df_sets.sort()
training_sets = df_sets[:int(len(df_sets)*.8)]
test_sets = df_sets[int(len(df_sets)*.8):]

In [ ]:
X_columns = [
    'flow_duration', 'Header_Length', 'Protocol Type', 'Duration',
       'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number',
       'rst_flag_number', 'psh_flag_number', 'ack_flag_number',
       'ece_flag_number', 'cwr_flag_number', 'ack_count',
       'syn_count', 'fin_count', 'urg_count', 'rst_count',
    'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP',
       'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min',
       'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue',
       'Radius', 'Covariance', 'Variance', 'Weight',
]
y_column = 'label'

## Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler = StandardScaler()

In [ ]:
for train_set in tqdm(training_sets):
    scaler.fit(pd.read_csv(datasetFolder + train_set)[X_columns])

100%|██████████| 135/135 [07:13<00:00,  3.21s/it]


## Training

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
# Initialize the Naive Bayes classifier
nb_model = GaussianNB()

# Train the model using the training data
for train_set in tqdm(training_sets):
    df_train = pd.read_csv(datasetFolder + train_set)
    X_train = scaler.transform(df_train[X_columns])
    y_train = df_train[y_column]  # assuming 'class' is the target column name
    nb_model.partial_fit(X_train, y_train, classes=np.unique(y_train))


100%|██████████| 135/135 [08:28<00:00,  3.77s/it]


## Evaluate

In [ ]:
# Evaluate the model using the test data
all_predictions = []
all_true_labels = []

for test_set in tqdm(test_sets):
    df_test = pd.read_csv(datasetFolder + test_set)
    X_test = scaler.transform(df_test[X_columns])
    y_test = df_test[y_column]  # assuming 'class' is the target column name
    predictions = nb_model.predict(X_test)

    all_predictions.extend(predictions)
    all_true_labels.extend(y_test)

# Calculate accuracy and other metrics
accuracy = accuracy_score(all_true_labels, all_predictions)
report = classification_report(all_true_labels, all_predictions)



100%|██████████| 34/34 [03:57<00:00,  6.99s/it]


In [ ]:
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

Accuracy: 0.7610609738088218
Classification Report:
                         precision    recall  f1-score   support

       Backdoor_Malware       0.00      0.04      0.00       728
          BenignTraffic       0.75      0.37      0.49    243322
       BrowserHijacking       0.00      0.19      0.01      1250
       CommandInjection       0.01      0.22      0.01      1154
 DDoS-ACK_Fragmentation       1.00      0.89      0.94     62723
        DDoS-HTTP_Flood       0.05      0.01      0.02      6450
        DDoS-ICMP_Flood       1.00      1.00      1.00   1594776
DDoS-ICMP_Fragmentation       0.99      0.97      0.98    100723
      DDoS-PSHACK_Flood       1.00      0.98      0.99    907911
       DDoS-RSTFINFlood       1.00      1.00      1.00    896126
         DDoS-SYN_Flood       0.64      0.91      0.75    900820
         DDoS-SlowLoris       0.16      0.87      0.27      5278
DDoS-SynonymousIP_Flood       0.72      0.69      0.71    795606
         DDoS-TCP_Flood       0.64   